In [26]:
import configparser as ConfigParser
import json 
import re


def load_patterns(fpath):
		config = ConfigParser.ConfigParser()
		with open(fpath) as f:
			config.readfp(f)

		for ind_type in config.sections():
			try:
				ind_pattern = config.get(ind_type, 'pattern')
			except:
				continue

			if ind_pattern:
				ind_regex = re.compile(ind_pattern)
				patterns[ind_type] = ind_regex
                
patterns = dict()

load_patterns('patterns.ini')

with open('new_tweets.json','r') as f:
    tweets = json.load(f)
    


def extract_tweet(s):
    t = s
    urls = list(map(lambda x:dict(url=x['expanded_url'],start=x['indices'][0],end=x['indices'][1]),t['entities']['urls']))
    mentions = list(map(lambda x:dict(screen_name=x['screen_name'],start=x['indices'][0],end=x['indices'][1]),t['entities']['user_mentions']))
    hashtags = list(map(lambda x:dict(tag=x['text'],start=x['indices'][0],end=x['indices'][1]),t['entities']['hashtags']))
    return dict(id=t['id'],text=t['text'], urls=urls,mentions=mentions,hashtags=hashtags,user=t['user'],retweet_count=t['retweet_count'],created_at=t['created_at'],likes=t['favorite_count'])
    




C:\Users\research\PycharmProjects\SAS-IOC-Workshop\venv\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  if __name__ == '__main__':


In [27]:
parsed_tweets = list(map(extract_tweet, tweets))

In [50]:
vt = []
for n in parsed_tweets:
    matches = []
    for j in n['hashtags']:
        for k,v in patterns.items():
            if v.search(j['tag']):
                matches.append(k)

    for j in n['urls']:
        #if '/file/' in j['url'].lower() or '/sample/' in j['url'].lower() or 'joe' in j['url'].lower():
        if (patterns['MD5'].search(j['url']) or patterns['SHA1'].search(j['url']) or patterns['SHA256'].search(j['url'])):
            hashes = ['MD5','SHA1','SHA256']
            for hash in hashes:
                if patterns[hash].findall(j['url']):
                    matches.append(''.join(patterns[hash].findall(j['url'])))
    if len(matches) > 0:
        for k,v in patterns.items():
                if v.search(n['text']) :
                    matches.append(k)     
        vt.append(dict(matches=matches,id=n['id'],tweet_id=n['user']['id'],screen_name=n['user']['screen_name'],mentions=n['mentions'],text=n['text'],url=n['urls'],hashtags=list(map(lambda x: x['tag'],n['hashtags']))))
print(vt)

[{'matches': ['2b5c31a6048574f9b8a0769ed1d8af362c5b354be3a31e6a1ae31c24ac127008'], 'id': 1115272145502965760, 'tweet_id': 73131445, 'screen_name': 'DFICyber', 'mentions': [{'screen_name': 'malwrhunterteam', 'start': 3, 'end': 19}], 'text': 'RT @malwrhunterteam: Turkish Aurora offline ransomware sample: https://t.co/IYn77AV1Sz\nExtension: .cryptoid\nNote names:\n@@_BENI_OKU_@@.txt…', 'url': [{'url': 'https://www.virustotal.com/en/file/2b5c31a6048574f9b8a0769ed1d8af362c5b354be3a31e6a1ae31c24ac127008/analysis/1554410364/', 'start': 63, 'end': 86}], 'hashtags': []}, {'matches': ['2b5c31a6048574f9b8a0769ed1d8af362c5b354be3a31e6a1ae31c24ac127008'], 'id': 1115266500812857344, 'tweet_id': 1351385155, 'screen_name': 'TheSicarioTR', 'mentions': [{'screen_name': 'malwrhunterteam', 'start': 3, 'end': 19}], 'text': 'RT @malwrhunterteam: Turkish Aurora offline ransomware sample: https://t.co/IYn77AV1Sz\nExtension: .cryptoid\nNote names:\n@@_BENI_OKU_@@.txt…', 'url': [{'url': 'https://www.virustotal.